In [66]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [117]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.hl1 = nn.ReLU()
        self.fc2 = nn.Linear(50, num_classes)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.hl1(x)
        x = self.fc2(x)
        return x

In [118]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [128]:
INPUT_SIZE = 784
NUM_CLASSES = 10
EPOCHS = 10
BATCH_SIZE = 64
LEARNING_RATE = 0.001

In [120]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [121]:
model = NN(input_size=INPUT_SIZE, num_classes=NUM_CLASSES).to(device)

In [122]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [129]:
for epoch in range(EPOCHS):
    for __, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        data = data.reshape(data.shape[0], -1)
        
        scores = model(data)
        loss = criterion(scores, targets)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()

In [124]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            
            x = x.reshape(x.shape[0], -1)
            
            scores = model(x)
            __, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
        print(f'Got {num_correct}/{num_samples} with accuracy: {float(num_correct)/float(num_samples)*100:.2f}%')   
        
    model.train()
            

In [130]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Got 59744/60000 with accuracy: 99.57%
Got 9724/10000 with accuracy: 97.24%
